**Text Loader**

In [1]:
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

text = load_text("dummy_data.txt")
print(text[:500])

In 2016, Dr. Amelia Reyes, an analytical and empathetic neuroscientist, joined MindWorks Research Institute as its lead data scientist. She worked closely with Professor Liam Chen, a methodical and calm researcher specializing in cognitive computing. Together, they developed a neural mapping algorithm that later became the foundation for the startup NeuroPath Analytics.

Marcus Tan, a decisive and visionary entrepreneur, founded GreenGrid Technologies in 2018 to promote sustainable energy soluti


**Text Splitter**

In [31]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

sentences = sent_tokenize(text)
print(f"Number of sentences: {len(sentences)}")
print(sentences[:5])


Number of sentences: 14
['In 2016, Dr. Amelia Reyes, an analytical and empathetic neuroscientist, joined MindWorks Research Institute as its lead data scientist.', 'She worked closely with Professor Liam Chen, a methodical and calm researcher specializing in cognitive computing.', 'Together, they developed a neural mapping algorithm that later became the foundation for the startup NeuroPath Analytics.', 'Marcus Tan, a decisive and visionary entrepreneur, founded GreenGrid Technologies in 2018 to promote sustainable energy solutions.', 'Under his leadership, the company partnered with EcoWave Labs, led by Dr. Sophia Mendoza, a pragmatic and detail-oriented environmental engineer.']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


**Entity Extraction Chain**

In [6]:
!pip install -U langchain langchain-community openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully

In [14]:
!pip install -qU langchain langchain-community langchain-core langchain-openai openai tiktoken


In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser


In [26]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [27]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize the JSON parser (no need for ResponseSchema anymore)
parser = JsonOutputParser()


In [33]:
prompt = ChatPromptTemplate.from_template(
"""
You are an expert information extractor for knowledge graphs.
Extract all named entities from the following text and categorize them as one of these types:
Person, Organization, Year.

Return your answer in valid JSON format as:
{{
  "entities": [
    {{"name": "Entity Name", "type": "Person/Organization/Year"}}
  ]
}}

Text:
\"\"\"{text}\"\"\"
""")


In [34]:
entity_chain = prompt | llm | parser

In [35]:
results = []

for i, sentence in enumerate(sentences[:5]):  # limit for testing
    response = entity_chain.invoke({"text": sentence})
    print(f"\nSentence {i+1}:\n", response)
    results.append(response)



Sentence 1:
 {'entities': [{'name': 'Amelia Reyes', 'type': 'Person'}, {'name': 'MindWorks Research Institute', 'type': 'Organization'}, {'name': '2016', 'type': 'Year'}]}

Sentence 2:
 {'entities': [{'name': 'Liam Chen', 'type': 'Person'}]}

Sentence 3:
 {'entities': [{'name': 'NeuroPath Analytics', 'type': 'Organization'}]}

Sentence 4:
 {'entities': [{'name': 'Marcus Tan', 'type': 'Person'}, {'name': 'GreenGrid Technologies', 'type': 'Organization'}, {'name': '2018', 'type': 'Year'}]}

Sentence 5:
 {'entities': [{'name': 'EcoWave Labs', 'type': 'Organization'}, {'name': 'Dr. Sophia Mendoza', 'type': 'Person'}]}


**Relationship Extraction Chain**

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# Initialize model and parser
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = JsonOutputParser()

# PROMPT (escaped curly braces)
rel_prompt = ChatPromptTemplate.from_template("""
You are an expert in semantic relationship extraction for knowledge graphs.

Given the following text, and the entities already detected, extract all meaningful relationships between them.
Possible relationship types include:
- works_at
- collaborated_with
- founded
- developed
- leads
- mentors

Return only valid JSON in this format:
{{
  "relationships": [
    {{"subject": "Entity1", "relation": "relation_type", "object": "Entity2"}}
  ]
}}

Text:
\"\"\"{text}\"\"\"

Detected entities:
{entities}
""")

In [40]:
rel_chain = rel_prompt | llm | parser


In [41]:
sentence = "In 2016, Dr. Amelia Reyes joined MindWorks Research Institute as its lead data scientist."
entities = [{'name': 'Amelia Reyes', 'type': 'Person'}, {'name': 'MindWorks Research Institute', 'type': 'Organization'}, {'name': '2016', 'type': 'Year'}]

result = rel_chain.invoke({"text": sentence, "entities": entities})
print(result)


{'relationships': [{'subject': 'Amelia Reyes', 'relation': 'works_at', 'object': 'MindWorks Research Institute'}]}


In [44]:
relationship_results = []

for i, sentence in enumerate(sentences[:5]):
    ents = results[i]['entities']  # assuming you stored them
    rels = rel_chain.invoke({"text": sentence, "entities": ents})
    relationship_results.append(rels)
    print(f"\nSentence {i+1}:\n", rels)



Sentence 1:
 {'relationships': [{'subject': 'Amelia Reyes', 'relation': 'works_at', 'object': 'MindWorks Research Institute'}, {'subject': 'Amelia Reyes', 'relation': 'leads', 'object': 'MindWorks Research Institute'}]}

Sentence 2:
 {'relationships': [{'subject': 'She', 'relation': 'collaborated_with', 'object': 'Liam Chen'}]}

Sentence 3:
 {'relationships': [{'subject': 'they', 'relation': 'developed', 'object': 'NeuroPath Analytics'}]}

Sentence 4:
 {'relationships': [{'subject': 'Marcus Tan', 'relation': 'founded', 'object': 'GreenGrid Technologies'}]}

Sentence 5:
 {'relationships': [{'subject': 'EcoWave Labs', 'relation': 'collaborated_with', 'object': 'Dr. Sophia Mendoza'}]}


**Personality Extraction Chain**

In [48]:
trait_prompt = ChatPromptTemplate.from_template("""
You are a psychologist AI trained to infer personality traits from text.
Extract the personality traits explicitly or implicitly described for any person in the given text.

Return only JSON in this format:
{{
  "personality_traits": [
    {{"person": "Person Name", "traits": ["trait1", "trait2", "..."]}}
  ]
}}

Text:
\"\"\"{text}\"\"\"
""")

In [49]:
trait_chain = trait_prompt | llm | parser

In [50]:
traits_result = trait_chain.invoke({"text": sentence})
print(traits_result)


{'personality_traits': [{'person': 'Dr. Sophia Mendoza', 'traits': ['pragmatic', 'detail-oriented']}]}


**Graph Assembly**

In [52]:
import networkx as nx

G = nx.DiGraph()

# Add entities
for ent_list in results:
    for e in ent_list['entities']:
        G.add_node(e['name'], type=e['type'])

# Add relationships
for rel_list in relationship_results:
    for r in rel_list['relationships']:
        G.add_edge(r['subject'], r['object'], relation=r['relation'])


In [45]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.7 MB/s eta 0:00:00


In [53]:
from pyvis.network import Network

net = Network(notebook=True, directed=True)
for n, data in G.nodes(data=True):
    label = f"{n}\n({data.get('type', '')})"
    net.add_node(n, label=label)
for u, v, data in G.edges(data=True):
    net.add_edge(u, v, label=data.get('relation', ''))

net.show("knowledge_graph.html")

knowledge_graph.html
